In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
findspark.find()

'C:\\spark-3.3.1-bin-hadoop2'

# method 1 reading csv file into spark session directly from local

In [4]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, cast, count, isnan, isnull, when


In [5]:
spark = SparkSession.builder.appName("diabet").master("local[2]").getOrCreate()

In [6]:
spark

In [8]:
#spark.sparkContext.setSystemProperty()

In [9]:
rawd = spark.read.csv("../drang/Downloads/diabetes.csv", header=True)
df = spark.range(0,20)
print(df.rdd.getNumPartitions())

2


In [10]:
rawd.rdd.getNumPartitions()

1

#  method 2 creating DB from csv local file and updating to server

In [11]:
#method 2 creating DB from csv local file and updating to server
import pandas as pd

data = pd.read_csv(r"C:\Users\drang\Downloads\diabetes.csv")


In [12]:
import mysql.connector

db_connection = mysql.connector.connect(user="root", password="Art8870496550!@#")
db_cursor = db_connection.cursor(buffered=True)
#db_cursor.execute("CREATE DATABASE DDB;")
db_cursor.execute("USE DDB;")



In [13]:
db_cursor.execute("CREATE TABLE diabete(Pregnancies INT, Glucose INT, \
                   BloodPressure INT, SkinThickness INT, BMI FLOAT, \
                   Insulin INT, DiabetesPedigreeFunction FLOAT, Age FLOAT, Outcome INT);")


ProgrammingError: 1050 (42S01): Table 'diabete' already exists

In [14]:
#create structure for sql
wtuples = list(data.itertuples(index=False, name=None))
tuples_string = ",".join(["(" + ",".join([str(w) for w in wt]) + ")" for wt in wtuples])

In [15]:
db_cursor.execute("INSERT INTO diabete(Pregnancies, Glucose, BloodPressure,\
                   SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome) VALUES " + tuples_string + ";")
db_cursor.execute("FLUSH TABLES;")

In [16]:
db_cursor.execute("SELECT * FROM diabete;")

In [17]:
spark.stop()

# method 2-2pt read data from mysql server into spark session

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.jars", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
.config("spark.driver.extraClassPath", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
.config("spark.executor.extraClassPath", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
.config("spark.executor.extraLibrary", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
    .master("local").appName("PySpark_MySQL_test").enableHiveSupport().getOrCreate()
#last 2 configs for cluster mode instead of local mode  
    
    
df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/DDB") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "diabete") \
    .option("user", "root").option("password", "Art8870496550!@#").load() 

In [19]:
df.show()

+-----------+-------+-------------+-------------+----+-------+------------------------+----+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness| BMI|Insulin|DiabetesPedigreeFunction| Age|Outcome|
+-----------+-------+-------------+-------------+----+-------+------------------------+----+-------+
|          6|    148|           72|           35|33.6|      0|                   0.627|50.0|      1|
|          1|     85|           66|           29|26.6|      0|                   0.351|31.0|      0|
|          8|    183|           64|            0|23.3|      0|                   0.672|32.0|      1|
|          1|     89|           66|           23|28.1|     94|                   0.167|21.0|      0|
|          0|    137|           40|           35|43.1|    168|                   2.288|33.0|      1|
|          5|    116|           74|            0|25.6|      0|                   0.201|30.0|      0|
|          3|     78|           50|           32|31.0|     88|                   0.248|26.0

+-----------+-------+-------------+-------------+----+-------+------------------------+----+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness| BMI|Insulin|DiabetesPedigreeFunction| Age|Outcome|
+-----------+-------+-------------+-------------+----+-------+------------------------+----+-------+
|          6|    148|           72|           35|33.6|      0|                   0.627|50.0|      1|
|          1|     85|           66|           29|26.6|      0|                   0.351|31.0|      0|
|          8|    183|           64|            0|23.3|      0|                   0.672|32.0|      1|
|          1|     89|           66|           23|28.1|     94|                   0.167|21.0|      0|
|          0|    137|           40|           35|43.1|    168|                   2.288|33.0|      1|
|          5|    116|           74|            0|25.6|      0|                   0.201|30.0|      0|
|          3|     78|           50|           32|31.0|     88|                   0.248|26.0

In [44]:
df.write.bucketBy(3,"age")
#df.rdd.collect()
df.show()

+-----------+-------+-------------+-------------+----+-------+------------------------+----+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness| BMI|Insulin|DiabetesPedigreeFunction| Age|Outcome|
+-----------+-------+-------------+-------------+----+-------+------------------------+----+-------+
|          6|    148|           72|           35|33.6|      0|                   0.627|50.0|      1|
|          1|     85|           66|           29|26.6|      0|                   0.351|31.0|      0|
|          8|    183|           64|            0|23.3|      0|                   0.672|32.0|      1|
|          1|     89|           66|           23|28.1|     94|                   0.167|21.0|      0|
|          0|    137|           40|           35|43.1|    168|                   2.288|33.0|      1|
|          5|    116|           74|            0|25.6|      0|                   0.201|30.0|      0|
|          3|     78|           50|           32|31.0|     88|                   0.248|26.0

In [45]:
columnname = rawd.columns

In [47]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: double (nullable = true)
 |-- Outcome: integer (nullable = true)



In [48]:
rawf = df.select(*(col(c).cast('float').alias(c) for c in rawd.columns))

In [49]:
rawf.printSchema()

root
 |-- Pregnancies: float (nullable = true)
 |-- Glucose: float (nullable = true)
 |-- BloodPressure: float (nullable = true)
 |-- SkinThickness: float (nullable = true)
 |-- Insulin: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- DiabetesPedigreeFunction: float (nullable = true)
 |-- Age: float (nullable = true)
 |-- Outcome: float (nullable = true)



In [50]:
rawf.select([count(when(col(c).isNull(),c)).alias(c) for c in rawf.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [51]:
#if theres null use imputer
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCol='Age', outputCol='newage')
model = imputer.fit(rawf)
data = model.transform(rawf)

In [52]:
data.rdd.getNumPartitions()
d = data.repartition(5)
d.rdd.getNumPartitions()
d

DataFrame[Pregnancies: float, Glucose: float, BloodPressure: float, SkinThickness: float, Insulin: float, BMI: float, DiabetesPedigreeFunction: float, Age: float, Outcome: float, newage: float]

In [53]:

colsi = rawf.drop(data.Outcome)

colsi = colsi.columns
colsi

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [54]:
from pyspark.ml.feature import VectorAssembler
n= VectorAssembler(inputCols=colsi, outputCol="feature")
newd = n.transform(rawf)
newd.show(truncate=False)

+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+-----------------------------------------------------------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI |DiabetesPedigreeFunction|Age |Outcome|feature                                                                |
+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+-----------------------------------------------------------------------+
|6.0        |148.0  |72.0         |35.0         |0.0    |33.6|0.627                   |50.0|1.0    |[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0]   |
|1.0        |85.0   |66.0         |29.0         |0.0    |26.6|0.351                   |31.0|0.0    |[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.35100001096725464,31.0]   |
|8.0        |183.0  |64.0         |0.0          |0.0    |23.3|0.672                   |32.0|1.0    |[8.0,183.0,64.0,0.0,0.0,23.29999923

In [55]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='feature', outputCol="scaledfeat")
model = scaler.fit(newd)
scaledd = model.transform(newd)


In [56]:
newdata = scaledd.select('scaledfeat','Outcome')

In [57]:
newdata.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|scaledfeat                                                                                                                                             |Outcome|
+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|[1.7815667561673982,4.631374347064541,3.7217532945376686,2.195196249533762,0.0,4.263931157322168,1.8933677423459543,4.253833663939351]                 |1.0    |
|[0.2969277926945664,2.659910942570851,3.4116071866595297,1.8188768924708312,0.0,3.3756123679198224,1.0599236453280207,2.6373768716423975]              |0.0    |
|[2.375422341556531,5.726631794005479,3.30822515070015,0.0,0.0,2.956833250721866,2.0292554353340746,2.7224535449211844]                                 |1.0    |
|[0.2969277926945664,2.78508

In [58]:
from pyspark.ml.feature import ChiSqSelector
chi = ChiSqSelector(numTopFeatures=6, outputCol="selectedcols", featuresCol="scaledfeat", labelCol="Outcome")
Sdata = chi.fit(newdata).transform(newdata)

In [59]:
Sdata.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------------------------------------------------------------------------------------------------------------------+
|scaledfeat                                                                                                                                             |Outcome|selectedcols                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------------------------------------------------------------------------------------------------------------------+
|[1.7815667561673982,4.631374347064541,3.7217532945376686,2.195196249533762,0.0,4.263931157322168,1.8933677423459543,4.253833663939351]                 |1.0    |[1.78156

In [60]:
columnname #3rd and 7th- BP & DIABETESPEDIGREE REMOVED BY CHISQ FEAT SELECTOR

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [61]:
Ssdata= Sdata.drop("scaledfeat")
Ssdata.show(truncate=False)

+-------+------------------------------------------------------------------------------------------------------------------+
|Outcome|selectedcols                                                                                                      |
+-------+------------------------------------------------------------------------------------------------------------------+
|1.0    |[1.7815667561673982,4.631374347064541,3.7217532945376686,2.195196249533762,0.0,4.263931157322168]                 |
|0.0    |[0.2969277926945664,2.659910942570851,3.4116071866595297,1.8188768924708312,0.0,3.3756123679198224]               |
|1.0    |[2.375422341556531,5.726631794005479,3.30822515070015,0.0,0.0,2.956833250721866]                                  |
|0.0    |[0.2969277926945664,2.785083222221244,3.4116071866595297,1.4425575354079005,0.816085934792367,3.565966446087695]  |
|1.0    |[0.0,4.28715057802596,2.067640719187594,2.195196249533762,1.4585365643097623,5.469506985718695]                   |


In [62]:
#traib test split
train, test = newdata.randomSplit([0.7,0.3])
test.count()
newdata.show()

+--------------------+-------+
|          scaledfeat|Outcome|
+--------------------+-------+
|[1.78156675616739...|    1.0|
|[0.29692779269456...|    0.0|
|[2.37542234155653...|    1.0|
|[0.29692779269456...|    0.0|
|[0.0,4.2871505780...|    1.0|
|[1.48463896347283...|    0.0|
|[0.89078337808369...|    1.0|
|[2.96927792694566...|    0.0|
|[0.59385558538913...|    1.0|
|[2.37542234155653...|    1.0|
|[1.18771117077826...|    0.0|
|[2.96927792694566...|    1.0|
|[2.96927792694566...|    0.0|
|[0.29692779269456...|    1.0|
|[1.48463896347283...|    1.0|
|[2.07849454886196...|    1.0|
|[0.0,3.6925822496...|    1.0|
|[2.07849454886196...|    1.0|
|[0.29692779269456...|    0.0|
|[0.29692779269456...|    1.0|
+--------------------+-------+
only showing top 20 rows



In [63]:
from pyspark.ml.classification import LogisticRegression, GBTClassifier, RandomForestClassifier

In [64]:
logreg = LogisticRegression(labelCol="Outcome", featuresCol="scaledfeat", maxIter=500, regParam=0.2, elasticNetParam=0.8)
model = logreg.fit(train)
preds = model.transform(test)
evals = model.evaluate(test)

In [65]:
preds.show()

print(evals.accuracy, evals.areaUnderROC)


+--------------------+-------+--------------------+--------------------+----------+
|          scaledfeat|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(8,[0,1,6,7],[0.5...|    0.0|[0.99718573517930...|[0.73050490091389...|       0.0|
|(8,[0,1,6,7],[0.5...|    0.0|[0.92031705914315...|[0.71510670428188...|       0.0|
|(8,[0,1,6,7],[0.5...|    0.0|[0.92031705914315...|[0.71510670428188...|       0.0|
|(8,[0,1,6,7],[0.5...|    0.0|[0.92031705914315...|[0.71510670428188...|       0.0|
|(8,[0,1,6,7],[0.8...|    0.0|[0.95106452955761...|[0.72132921311564...|       0.0|
|(8,[0,1,6,7],[2.0...|    0.0|[0.75889283946723...|[0.68111330900451...|       0.0|
|(8,[0,1,6,7],[2.0...|    0.0|[0.75889283946723...|[0.68111330900451...|       0.0|
|(8,[0,1,6,7],[2.9...|    1.0|[0.68202416343108...|[0.66419031987256...|       0.0|
|(8,[1,5,6,7],[2.2...|    0.0|[1.00487260278292...|[0.73201551122081...|    

In [66]:
"""from pyspark.mllib.evaluation import BinaryClassificationMetrics
#convert to rdd cox of mllib pkg
predsa = preds.select('Outcome','prediction').rdd.map(lambda row: (float(row['probability'][1]), float(row['Outcome'])))
metrics = BinaryClassificationMetrics(predsa)"""

"from pyspark.mllib.evaluation import BinaryClassificationMetrics\n#convert to rdd cox of mllib pkg\npredsa = preds.select('Outcome','prediction').rdd.map(lambda row: (float(row['probability'][1]), float(row['Outcome'])))\nmetrics = BinaryClassificationMetrics(predsa)"

In [67]:
"""print("AUC", metrics.areaUnderPR)"""

'print("AUC", metrics.areaUnderPR)'

In [68]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

In [69]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
metric = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Outcome", metricName="areaUnderROC")
lrroc=metric.evaluate(predy)
lrroc

0.7718205337690633

In [70]:
metrics = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="Outcome", metricName="accuracy")
lracc = metrics.evaluate(predy)
lracc

0.6494845360824743

In [71]:

rf = RandomForestClassifier(labelCol="Outcome", featuresCol="scaledfeat", numTrees=20)
rfmod = rf.fit(train)
pred = rfmod.transform(test)

In [72]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

In [73]:
rfroc = metric.evaluate(predy)

In [74]:
rfacc = metrics.evaluate(predy)

In [75]:
ml = GBTClassifier(labelCol="Outcome", featuresCol=("scaledfeat"), maxDepth=10, maxIter=100)
model = ml.fit(train)


In [76]:
preds = model.transform(test)

#convert SparseVector -> DenseVector  which error occured during chisq selector

In [77]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

gbtroc=metric.evaluate(predy)

In [78]:
gbtacc = metrics.evaluate(predy)

In [79]:
roc = [lrroc, rfroc, gbtroc]
acc = [lracc, rfacc, gbtacc]
print(f"lrroc:{lrroc},  rfroc:{rfroc}, gbtroc:{gbtroc}")
print(f"lracc:{lracc},  rfacc:{rfacc}, gbtacc:{gbtacc}")

lrroc:0.7718205337690633,  rfroc:0.7718205337690633, gbtroc:1.0
lracc:0.6494845360824743,  rfacc:0.6494845360824743, gbtacc:1.0


In [80]:
from pyspark.mllib.stat import Statistics
n = newdata.drop("Outcome")
n = newdata.select(newdata.scaledfeat).rdd.map(lambda row : row[0:])
summary = Statistics.colStats(n)

summary.mean() #variance, norm

array([1.14170283, 3.78316102, 3.57213203, 1.28804447, 0.69280035,
       4.05994515, 1.42493685, 2.82802395])

In [81]:
correlat = Statistics.corr(n, method='pearson')
import pandas as pd
pddf= pd.DataFrame(correlat, columns=columnname[:-1], index=columnname[:-1])
pddf


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000


In [82]:
from pyspark.ml.stat import Correlation
n = newd.drop('Outcome')
n = n.drop('features')
summary = Correlation()
print(n)
columnname=columnname[:-1]
a=summary.corr(dataset=n.feature, column=columnname) #variance, norm
a

DataFrame[Pregnancies: float, Glucose: float, BloodPressure: float, SkinThickness: float, Insulin: float, BMI: float, DiabetesPedigreeFunction: float, Age: float, feature: vector]


Traceback (most recent call last):
  File "C:\spark-3.3.1-bin-hadoop2\python\pyspark\serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "C:\spark-3.3.1-bin-hadoop2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "C:\spark-3.3.1-bin-hadoop2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object

In [83]:
n.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+----+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction| Age|             feature|
+-----------+-------+-------------+-------------+-------+----+------------------------+----+--------------------+
|        6.0|  148.0|         72.0|         35.0|    0.0|33.6|                   0.627|50.0|[6.0,148.0,72.0,3...|
|        1.0|   85.0|         66.0|         29.0|    0.0|26.6|                   0.351|31.0|[1.0,85.0,66.0,29...|
|        8.0|  183.0|         64.0|          0.0|    0.0|23.3|                   0.672|32.0|[8.0,183.0,64.0,0...|
|        1.0|   89.0|         66.0|         23.0|   94.0|28.1|                   0.167|21.0|[1.0,89.0,66.0,23...|
|        0.0|  137.0|         40.0|         35.0|  168.0|43.1|                   2.288|33.0|[0.0,137.0,40.0,3...|
|        5.0|  116.0|         74.0|          0.0|    0.0|25.6|                   0.201|3